### Generate recommendations by linking structured & unstructured text

## Import the necesssary libraries.

### 1. SETUP
To prepare your environment, you need to install some packages and enter credentials for the Watson services.



In [1]:
!pip install nltk

Requirement not upgraded as not directly required: nltk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from nltk)


In [2]:
!pip install watson-developer-cloud==1.5

Requirement not upgraded as not directly required: watson-developer-cloud==1.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: python-dateutil>=2.5.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: requests<3.0,>=2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: pyOpenSSL>=16.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: Twisted>=13.2.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as not directly required: service-identity>=17.0.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-developer-cloud==1.5)
Requirement not upgraded as no

In [3]:
!pip install PyPDF2 

Requirement not upgraded as not directly required: PyPDF2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [4]:
!pip install mammoth

Requirement not upgraded as not directly required: mammoth in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: cobble<0.2,>=0.1.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from mammoth)


In [5]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/dsxuser/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/dsxuser/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/dsxuser/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/dsxuser/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/dsxuser/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/dsxuser/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading p

True

### 2. Import packages and libraries
Import the packages and libraries that you'll use:

In [6]:
import pandas as pd
import json
import re
import nltk
from botocore.client import Config
import ibm_boto3
import zipfile
from io import BytesIO
from IPython.display import display, HTML

import os, sys, glob, mammoth

from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, SemanticRolesOptions, RelationsOptions, KeywordsOptions

import PyPDF2 


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


### Load the Configuration file.

In [7]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
    'FILE': 'sample_config.txt'
}

In [8]:
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

    
def get_docx_file():
    '''Retrieve file '''
    docx_files=[]
    zip_ref = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
    paths = zip_ref.namelist()
    for path in paths:
        file=zip_ref.extract(path)
        docx_files.append(file)
    return docx_files



def load_string(fileobject):
    '''Load the file contents into a Python string'''

    text = fileobject.read()
    return text

def load_df(fileobject,sheetname):
    '''Load file contents into a Pandas dataframe'''
    excelFile = pd.ExcelFile(fileobject)
    df = excelFile.parse(sheetname)
    return df

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp

In [9]:
config_classification_json= json.loads(load_string(get_file(credentials_1['FILE'])).decode("utf-8"))
config_classification_json

{'configuration': {'classification': {'stages': [{'name': 'Base Tagging',
     'steps': [{'keywords': [{'tag': 'Recruitment', 'text': 'Job profile'},
        {'tag': 'Skill', 'text': 'User Experience'}],
       'type': 'keywords'},
      {'d_regex': [{'pattern': '(\\d+/\\d+/\\d+)', 'tag': 'Date'}],
       'type': 'd_regex'},
      {'d_regex': [{'pattern': '\\b[\\w.-]+?@\\w+?\\.\\w+?\\b',
         'tag': 'Email'}],
       'type': 'd_regex'},
      {'d_regex': [{'pattern': '[0-9]{10}', 'tag': 'PhoneNumber'}],
       'type': 'd_regex'},
      {'chunk': [{'pattern': 'NP:{<DT>?<JJ>*<NN>}', 'tag': 'NP'},
        {'pattern': 'NAME:{<NNP>+}', 'tag': 'NAME'}],
       'type': 'chunking'}]},
    {'name': 'Domain Tagging',
     'steps': [{'d_regex': [{'pattern': '[0-9]{4}', 'tag': 'Year'}],
       'type': 'd_regex'}]}]}}}

### Load the structured data from Data.csv. Insert it as pandas dataframe and change the name from df_data_1 to stu_df

In [10]:


stu_df = pd.read_csv(body)
stu_df.head()

,Name,Handphone,Email,Education,Experience in Months,Skills,Applied Before,Comments
0,C1,1010101010,C1@yahoo.com,Masters of Design,54,"User Experience Design, Interaction Design, In...",Yes,Did not accept offer
1,C2,9008007000,C2@hotmail.com,"B.Tech, Civil Engineering",24,SAP Functional Consultant,Yes,Did not attend interview
2,C3,1009191000,C3@gmail.com,"Master's degree, Computer Science",32,Software Developer,No,NaN
3,C4,8007006000,C4@yahoo.com,"Master of Technology (MTech), Information Tech...",50,Software Engineer,Yes,Ex employee
4,C5,6007008000,C5@yahoo.com,"Master of Technology (M.Tech.), Information Te...",25,Devops/Software testing,No,NaN


### Insert the data_files.zip as streaming body object and change its name to streaming_body_1.

### 3. Add your service credentials from Bluemix for the Watson services.
You must create a Watson Natural Language Understanding service on Bluemix. Create a service for Natural Language Understanding (NLU). Insert the username and password values for your NLU in the following cell. Do not change the values of the version fields.

Run the cell.

In [12]:
natural_language_understanding = NaturalLanguageUnderstandingV1(
   username= "",
  password= "",
  version='2017-02-27')


### 4. Functions for  Watson Text Classification
Write the classification related utility functions in a modularalized form with augmentation. 



In [13]:
def analyze_using_NLU(text_content):
    '''
    Call Watson Natural Language Understanding service to obtain analysis results.
    '''
    response = natural_language_understanding.analyze(
        text= text_content,
        features=Features(
        entities=EntitiesOptions(),
        relations=RelationsOptions(),
        keywords= KeywordsOptions())
    )
    return response

In [14]:
def split_sentences(text):
    """ Split text into sentences.
    """
    sentence_delimiters = re.compile(u'[\\[\\]\n.!?]')
    sentences = sentence_delimiters.split(text)
    return sentences

def split_into_tokens(text):
    """ Split text into tokens.
    """
    tokens = nltk.word_tokenize(text)
    return tokens
    
def POS_tagging(text):
    """ Generate Part of speech tagging of the text.
    """
    POSofText = nltk.tag.pos_tag(text)
    return POSofText

def keyword_tagging(tag,tagtext,text):
    """ Tag the text matching keywords.
    """
    if (text.lower().find(tagtext.lower()) != -1):
        return text[text.lower().find(tagtext.lower()):text.lower().find(tagtext.lower())+len(tagtext)]
    else:
        return 'UNKNOWN'
    
def regex_tagging(tag,regex,text):
    """ Tag the text matching REGEX.
    """    
    p = re.compile(regex, re.IGNORECASE)
    matchtext = p.findall(text)
    regex_list=[]    
    if (len(matchtext)>0):
        for regword in matchtext:
            regex_list.append(regword)
    return regex_list

def chunk_tagging(tag,chunk,text):
    """ Tag the text using chunking.
    """
    parsed_cp = nltk.RegexpParser(chunk)
    pos_cp = parsed_cp.parse(text)
    chunk_list=[]
    for root in pos_cp:
        if isinstance(root, nltk.tree.Tree):               
            if root.label() == tag:
                chunk_word = ''
                for child_root in root:
                    chunk_word = chunk_word +' '+ child_root[0]
                chunk_list.append(chunk_word)
    return chunk_list
    
def augument_NLUResponse(responsejson,updateType,text,tag):
    """ Update the NLU response JSON with augumented classifications.
    """
    if(updateType == 'keyword'):
        if not any(d.get('text', None) == text for d in responsejson['keywords']):
            responsejson['keywords'].append({"text":text,"relevance":0.5})
    else:
        if not any(d.get('text', None) == text for d in responsejson['entities']):
            responsejson['entities'].append({"type":tag,"text":text,"relevance":0.5,"count":1})        
    

def classify_text(text, config):
    """ Perform augumented classification of the text.
    """
    
    response = analyze_using_NLU(text)
    responsejson = response
    
    sentenceList = split_sentences(text)
    
    tokens = split_into_tokens(text)
    
    postags = POS_tagging(tokens)
    
    configjson = config
    
    for stages in configjson['configuration']['classification']['stages']:
        for steps in stages['steps']:
            if (steps['type'] == 'keywords'):
                for keyword in steps['keywords']:
                    for word in sentenceList:
                        wordtag = keyword_tagging(keyword['tag'],keyword['text'],word)
                        if(wordtag != 'UNKNOWN'):
                            augument_NLUResponse(responsejson,'entities',wordtag,keyword['tag'])
            elif(steps['type'] == 'd_regex'):
                for regex in steps['d_regex']:
                    for word in sentenceList:
                        regextags = regex_tagging(regex['tag'],regex['pattern'],word)
                        if (len(regextags)>0):
                            for words in regextags:
                                augument_NLUResponse(responsejson,'entities',words,regex['tag'])
            elif(steps['type'] == 'chunking'):
                for chunk in steps['chunk']:
                    chunktags = chunk_tagging(chunk['tag'],chunk['pattern'],postags)
                    if (len(chunktags)>0):
                        for words in chunktags:
                            augument_NLUResponse(responsejson,'entities',words,chunk['tag'])
            else:
                print('UNKNOWN STEP')
    
    return responsejson

def replace_unicode_strings(response):
    """ Convert dict with unicode strings to strings.
    """
    if isinstance(response, dict):
        return {replace_unicode_strings(key): replace_unicode_strings(value) for key, value in response.iteritems()}
    elif isinstance(response, list):
        return [replace_unicode_strings(element) for element in response]
    elif isinstance(response, unicode):
        return response.encode('utf-8')
    else:
        return response

### 4. Extracting Requirements From the Job Description.

In [15]:
job_description_1 = "I need a candidate with User Experience Design skills and experience should be more than 24 months. "
job_description_2 = "I need a candidate with Machine Learning Expert skills and experience should be more than 27 months."
job_description = [job_description_1, job_description_2]

In [16]:
def getrequirements(job_description):
    requirement_jd=[]
    for i in job_description:
        requirement_jd.append(classify_text(i,config_classification_json))
    return requirement_jd

def getRequiredCandidateEntityList(requirement_jd):
    RequiredCandidateEntityList =[]
    text_type=''
    text_value = ''
    entity_dict={}
    for i in requirement_jd:
        entity_dict={}
        for k in i['entities']:
            for key1, value1 in k.items():
                if(key1=='type'):
                    text_type = value1
                if(key1=='text'):
                    text_value = value1
            entity_dict[text_type] = text_value
        RequiredCandidateEntityList.append(entity_dict)
    return RequiredCandidateEntityList

def getskills_matching_candidates(RequiredCandidateEntityList):
    '''
    Filtering the Candidates matching with the required skills.
    '''
    skills_matching_candidates =[]
    row_list = []
    for i in RequiredCandidateEntityList:
        requirement_1= i['NAME'].lstrip()
        requirement_2= i['Quantity'].lstrip()
        for index, row in stu_df.iterrows():
            if '/' or ',' in row['Skills']:
                if requirement_1 in list(re.split('\/|,',row['Skills'])):
                    row_list.append(row)
                    skills_matching_candidates.append(row['Name'])
            else:
                if(requirement_1 in row['Skills']):
                    row_list.append(row)
                    skills_matching_candidates.append(row['Name'])
    return row_list

In [17]:
requirement_jd = getrequirements(job_description)
requirement_jd

[{'entities': [{'count': 1,
    'relevance': 0.33,
    'text': '24 months',
    'type': 'Quantity'},
   {'count': 1, 'relevance': 0.5, 'text': 'User Experience', 'type': 'Skill'},
   {'count': 1, 'relevance': 0.5, 'text': ' a candidate', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' experience', 'type': 'NP'},
   {'count': 1,
    'relevance': 0.5,
    'text': ' User Experience Design',
    'type': 'NAME'}],
  'keywords': [{'relevance': 0.976375, 'text': 'User Experience Design'},
   {'relevance': 0.723048, 'text': 'candidate'},
   {'relevance': 0.350848, 'text': 'skills'}],
  'language': 'en',
  'relations': [],
  'usage': {'features': 3, 'text_characters': 100, 'text_units': 1}},
 {'entities': [{'count': 1,
    'relevance': 0.01,
    'text': '27 months',
    'type': 'Quantity'},
   {'count': 1, 'relevance': 0.5, 'text': ' a candidate', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' experience', 'type': 'NP'},
   {'count': 1,
    'relevance': 0.5,
    'text': '

In [18]:
RequiredCandidateEntityList = getRequiredCandidateEntityList(requirement_jd)
RequiredCandidateEntityList

[{'NAME': ' User Experience Design',
  'NP': ' experience',
  'Quantity': '24 months',
  'Skill': 'User Experience'},
 {'NAME': ' Machine Learning Expert',
  'NP': ' experience',
  'Quantity': '27 months'}]

In [19]:
row_list = getskills_matching_candidates(RequiredCandidateEntityList)
row_list
filtered_dataframe = pd.DataFrame(row_list)
filtered_dataframe


,Name,Handphone,Email,Education,Experience in Months,Skills,Applied Before,Comments
0,C1,1010101010,C1@yahoo.com,Masters of Design,54,"User Experience Design, Interaction Design, In...",Yes,Did not accept offer
9,C10,8765432100,C10@hotmail.com,"Master of Design, Design and Visual Communication",60,User Experience Design/Senior Product Designer,No,NaN
13,C14,1234561234,C14@gmail.com,"Bachelor of Engineering (BE), Information Tech...",55,Machine Learning Expert,Yes,Analytical horsepower is good


### 5. Processing the resumes. Check for the current directory and run the below cell

In [20]:
cd 

/home/dsxuser


#### Change the directory by running the below cell

In [21]:
cd /home/dsxuser/work

/home/dsxuser/work


### Use the below command to check the list of documents in the directory

In [22]:
ls

C10.pdf   C12.docx  C14.docx  C2.pdf  C4.pdf  C7.pdf  C9.pdf
C11.docx  C13.docx  C1.pdf    C3.pdf  C6.pdf  C8.pdf


### Unzip the data_files.zip

In [23]:
zip_ref = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
paths = zip_ref.namelist()

for path in paths:
    factsheet=zip_ref.extract(path)

path = '/home/dsxuser/work/'

In [24]:
print(paths)

['C6.pdf', 'C7.pdf', 'C8.pdf', 'C9.pdf', 'C10.pdf', 'C11.docx', 'C12.docx', 'C13.docx', 'C14.docx', 'C1.pdf', 'C2.pdf', 'C3.pdf', 'C4.pdf']


In [25]:

def extractingTextfromresumes():
    '''Extracting Text from the pool of resumes(processing word docs and pdfs)
    '''


    matching_candidates_text = []
    filenames = glob.glob(path+'/*.pdf')
    filenames_docx= glob.glob(path+'/*.docx')

    for filename in filenames:
        print(filename)
        pdfFileObj = open(filename,'rb')
        #The pdfReader variable is a readable object that will be parsed
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        #discerning the number of pages will allow us to parse through all #the pages
        num_pages = pdfReader.numPages
        count = 0
        text = ""
        #The while loop will read each page
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        
       
        matching_candidates_text.append(text)

    for filename in filenames_docx:
        print(filename)
        with open(filename, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value # The raw text
            messages = result.messages # Any messages
            matching_candidates_text.append(text)
            
    return matching_candidates_text

def processTheTextWithWatsonNLU(matching_candidates_text):
    '''
    Process the text with Watson NLU
    '''
    NLU_Results_Matched_Candidates = []
    for text in matching_candidates_text:
        json = classify_text(text,config_classification_json)
        NLU_Results_Matched_Candidates.append(json)
    return NLU_Results_Matched_Candidates


def unstructuredTexttoadataframe(NLU_Results_Matched_Candidates):
    '''
    Convert the unstructured text(entities in the result of NLU) to a dataframe
    '''
    matchedCandidateEntityList =[]
    entity_dict={}
    text_type=''
    text_value = ''
    for i in NLU_Results_Matched_Candidates:
        entity_dict={}
        for k in i['entities']:
            for key1, value1 in k.items():
                if(key1=='type'):
                    text_type = value1
                if(key1=='text'):
                    text_value = value1
            entity_dict[text_type] = text_value
        matchedCandidateEntityList.append(entity_dict)
    return matchedCandidateEntityList

In [26]:
matching_candidates_text = extractingTextfromresumes()
matching_candidates_text

/home/dsxuser/work/C6.pdf
/home/dsxuser/work/C9.pdf
/home/dsxuser/work/C8.pdf
/home/dsxuser/work/C2.pdf
/home/dsxuser/work/C4.pdf
/home/dsxuser/work/C1.pdf
/home/dsxuser/work/C7.pdf
/home/dsxuser/work/C10.pdf
/home/dsxuser/work/C3.pdf
/home/dsxuser/work/C12.docx
/home/dsxuser/work/C14.docx
/home/dsxuser/work/C13.docx
/home/dsxuser/work/C11.docx


['C6\n \n \nData Scientist \n \nHandphone: 1002003000\n \nEmail: \nC6\n@hotmail.com\n \n \n \n \nSummary\n \n \nWorking on challenging data and scalability problems as part of the Display\n-\nTargeting and Affiliate\n-\nMarketing Channels.\n \n \nRecommendation Systems:\n \n \n-\n \nBuild recommendation models.\n \n \nForecasting:\n \n \n-\n \nBuild models for Revenue Forecasting.\n \n \n-----------------------------------------------------------------------------------------------------------------------------\n----\n \nExperience\n \n \nSr. Engineer \n \nIndia November 2017 \n-\n \nPresent\n \n \nEngineer III\n \n \nNovember 2015 \n-\n \nOctober 2016 \n \n \nEngineer II \n \n \nJuly 2014 \n-\n \nOctober 2014\n \n \n \nEducation\n \n\n \n \n \nB TECH, CSE\n \n \n \n \n',
 'C9\n \n \nData Scientist\n \nHandphone:\n \n1234567891\n \nEmail:\n \nC9\n@gmail.com\n \n \n \n \nSummary\n \n \nHaving good experience in Data Extraction, Data Mining using SQL/DB2, Advancd Excel and data \nanalysi

In [27]:
NLU_Results_Matched_Candidates = processTheTextWithWatsonNLU(matching_candidates_text)

In [28]:
NLU_Results_Matched_Candidates

[{'entities': [{'count': 2,
    'relevance': 0.968639,
    'text': 'Engineer',
    'type': 'JobTitle'},
   {'count': 1,
    'relevance': 0.692875,
    'text': 'Sr. Engineer',
    'type': 'JobTitle'},
   {'count': 1,
    'relevance': 0.467827,
    'text': 'Scientist',
    'type': 'JobTitle'},
   {'count': 1,
    'relevance': 0.467827,
    'text': '@hotmail',
    'type': 'TwitterHandle'},
   {'count': 1, 'relevance': 0.5, 'text': '1002003000', 'type': 'PhoneNumber'},
   {'count': 1, 'relevance': 0.5, 'text': ' Email', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' hotmail.com', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' scalability', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' part', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' Targeting', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' Marketing', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5, 'text': ' recommendation', 'type': 'NP'},
   {'count': 1, 'relevance': 0.5,

In [29]:
matchedCandidateEntityList = unstructuredTexttoadataframe(NLU_Results_Matched_Candidates)
matchedCandidateEntityList

[{'JobTitle': 'Scientist',
  'NAME': ' CSE',
  'NP': ' Experience',
  'PhoneNumber': '1002003000',
  'TwitterHandle': '@hotmail',
  'Year': '2014'},
 {'JobTitle': 'consultant',
  'NAME': ' Computer Science',
  'NP': ' consultant',
  'Organization': 'Saint Soldier Public School',
  'Person': 'Banasthali Vidyapith',
  'PhoneNumber': '1234567891',
  'TwitterHandle': '@gmail',
  'Year': '2012'},
 {'Company': 'Mechanical Engineering',
  'JobTitle': 'Editor',
  'NAME': ' Mechanical Engineering',
  'NP': ' Education',
  'Organization': 'K.J. Somaiya Institute',
  'PhoneNumber': '1001001000',
  'PrintMedia': 'f Management Studies and Research',
  'TwitterHandle': '@hotmail',
  'Year': '2012'},
 {'Company': 'IIM Sambalpur',
  'EmailAddress': 'DEF@hotmail.com',
  'JobTitle': 'Coordinator',
  'Location': 'Trivandrum',
  'NAME': ' Math',
  'NP': ' hemistry',
  'Organization': 'College of Engineering',
  'Person': 'hemistry',
  'PhoneNumber': '9008007000',
  'Quantity': '2 years',
  'Year': '2015'}

In [30]:
resume_df = pd.DataFrame(matchedCandidateEntityList)
resume_df


,Company,EmailAddress,JobTitle,Location,NAME,NP,Organization,Person,PhoneNumber,PrintMedia,Quantity,Skill,TwitterHandle,Year
0,NaN,NaN,Scientist,NaN,CSE,Experience,NaN,NaN,1002003000,NaN,NaN,NaN,@hotmail,2014
1,NaN,NaN,consultant,NaN,Computer Science,consultant,Saint Soldier Public School,Banasthali Vidyapith,1234567891,NaN,NaN,NaN,@gmail,2012
2,Mechanical Engineering,NaN,Editor,NaN,Mechanical Engineering,Education,K.J. Somaiya Institute,NaN,1001001000,f Management Studies and Research,NaN,NaN,@hotmail,2012
3,IIM Sambalpur,DEF@hotmail.com,Coordinator,Trivandrum,Math,hemistry,College of Engineering,hemistry,9008007000,NaN,2 years,NaN,NaN,2015
4,NaN,C4@yahoo.com,Intern,Bangalore,Information Technology Page,manual testing,International Institute of Information,NaN,8007006000,NaN,NaN,NaN,NaN,2013
5,NaN,NaN,Intern,NaN,Computer Science,Present Retail Experience,National Institute of Design,NaN,1010101010,NaN,NaN,User Experience,@yahoo,2011
6,NaN,NaN,Trainee,Nagpur,Computer Science,BTech,Institute of Management Technology,NaN,7891234567,NaN,NaN,NaN,@yahoo,2015
7,NaN,NaN,Customer Relationship Management,Bombay,Visual Communication Page,Experience,Indian Institute of Technology,NaN,8765432100,NaN,NaN,NaN,@hotmail,2012
8,NaN,C3@gmail.com,Communications Engineering,Bangalore,Communications Engineering,degree,International Institute of,NaN,1009191000,NaN,NaN,NaN,NaN,2016
9,NaN,C12@yahoo.com,Trainee,NaN,Technology,BTech,Institute of Management Technology,NaN,9879879870,Natural Language Processing Expert,NaN,NaN,NaN,2016


### 6. Recommendation

In [31]:
def Recommendation(resume_df):
    recommendation=[]
    display(HTML('<!DOCTYPE html><html><title>W3.CSS</title><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css"><h2>Recommendation</h2></html>'))
    for index, row in resume_df.iterrows():
        if (int(row['PhoneNumber']) in list(filtered_dataframe['Handphone'])):
            applied_before = filtered_dataframe[filtered_dataframe['Handphone'] == int(row['PhoneNumber'])]['Applied Before'].iloc[0]
            comments = filtered_dataframe[filtered_dataframe['Handphone'] == int(row['PhoneNumber'])]['Comments'].iloc[0]

            name = filtered_dataframe[filtered_dataframe['Handphone'] == int(row['PhoneNumber'])]['Name'].iloc[0]

            if(applied_before.lower() == 'yes'):
                    print_card = "Candidate "+ name +" "+comments
                    display(HTML('<!DOCTYPE html><html><title>W3.CSS</title><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css"><body><div class="w3-container"><div class="w3-panel w3-card w3-red"><p>'+ print_card +'</p></div></div></body></html>'))
            

In [32]:
Recommendation(resume_df)

#### We can observe that for the first requirement, C1 & C10 were eligible however as per recommendation C1 had applied before and did not accept offer which makes C10 ideal candidate to accept the offer if selected. For the second requirement, C11 & C14 are eligible however the recommendation is to select C11 given the post graduate degree of specialization who will be a better fit for the role. 